In [ ]:
import os, sys

import time

import numpy as np
import tensorflow as tf
import re

import tflib as lib
import tflib.save_images
import tflib.plot

from sklearn.decomposition import TruncatedSVD
from keras_radam.training import RAdamOptimizer


LAMBDA = 1000.0 # Gradient penalty lambda hyperparameter --- it regulates how strongly we enforce lipschitzness
LAMBDA_2 = 5.0 # Constant for consistency term as in https://openreview.net/pdf?id=SJx9GQb0-
CRITIC_ITERS = 5 # How many critic iterations per generator iteration
BATCH_SIZE = 40 # Batch size
ITERS = 3000 # How many generator iterations to train for
num_epochs = 100 # number of iterations of alternating scheme
OUTPUT_DIM = 400 # output dimension
Lambda = 100.0 # the main lambda --- constant in front of the second term that enforces low-dimensionality of support 
Dimension = 10 # Hidden dimensionality of data --- rank of matrix L
GAMMA = 1/OUTPUT_DIM # parameter of gaussian kernel M(x,y) = exp(-GAMMA*(x-y)^2)
delta = 0.1 # regulates percentage of outliers
MODE = 'neutral' # case I - neutral, case II-something else

SIZE_COV = 10000*OUTPUT_DIM # how many times we sample fake and real to estimate matrix M_ф from step (**)
TIMES_COV = int(SIZE_COV/BATCH_SIZE)
SIZE_COV = TIMES_COV*BATCH_SIZE


In [ ]:
# here I generate matrix X
num = 400
A = np.random.normal(loc=0.0, scale=1.0, size=(int(num*(1-delta)), Dimension))
B = np.random.normal(loc=0.0, scale=1.0, size=(Dimension, OUTPUT_DIM))
if MODE == 'neutral':
    C = np.random.normal(loc=0.0, scale=1.0, size=(int(num*delta), OUTPUT_DIM))
else:
    C = np.tile(np.random.normal(loc=0.0, scale=1.0, size=(1, OUTPUT_DIM)), (int(num*delta), 1))
images = np.concatenate((np.matmul(A, B), C), axis = 0)
print(images.shape)
u, s, vh = np.linalg.svd(np.matmul(A, B), full_matrices=True)
D = vh[0:Dimension]
P_correct = np.matmul(np.transpose(D), D)

u, s, vh = np.linalg.svd(images, full_matrices=True)
D = vh[0:Dimension]
P_svd = np.matmul(np.transpose(D), D)
DIFF = P_svd - P_correct
dist = np.sqrt(np.sum(DIFF*DIFF))
print("Frobenius distance from SVD till correct %0.4f" %dist)

L1 = np.sum(np.sqrt(np.sum(np.square(images-np.matmul(images, P_correct)), axis=1)))
print("L12 distance till correct %0.4f" %L1)


In [ ]:
def get_vars_from_scope (scope = None):
    ret = []
    for var in tf.global_variables():
        if re.search(scope, var.name):
            ret.append(var)
    return ret

init_scale = 0.05
highway_size = OUTPUT_DIM
def highway_layer(highway_inputs):
    transf_weights = tf.get_variable('transf_weights', 
        [highway_size, highway_size],
        initializer=tf.zeros_initializer(),
        dtype=tf.float32)
    transf_biases = tf.get_variable('transf_biases', [highway_size],
        initializer=tf.zeros_initializer(),
        dtype=tf.float32)
    highw_weights = tf.get_variable('highw_weights', 
        [highway_size, highway_size],
        initializer=tf.initializers.identity(),
        dtype=tf.float32)
    highw_biases = tf.get_variable('highw_biases', [highway_size],
        initializer=tf.zeros_initializer(),
        dtype=tf.float32)
    transf_gate = tf.nn.sigmoid(tf.matmul(highway_inputs, transf_weights)         + transf_biases)
    highw_output = tf.multiply(transf_gate, 
        tf.nn.relu(tf.matmul(highway_inputs, highw_weights) + highw_biases)) \
        + tf.multiply(tf.ones([highway_size], dtype=tf.float32) - transf_gate, 
        highway_inputs)
    return highw_output

def Generator(n_samples, noise=None):
    if noise is None:
        noise = tf.random.uniform(shape=[n_samples], minval=0, maxval=OUTPUT_DIM, dtype=tf.dtypes.int32)
        output = tf.one_hot(noise, OUTPUT_DIM, axis=-1)
        noise1 = tf.random_normal([n_samples, OUTPUT_DIM])
    with tf.variable_scope('Generator', reuse=tf.AUTO_REUSE) as scope:
        output = tf.layers.dense(output, OUTPUT_DIM, activation=None)
        output1 = tf.layers.dense(noise1, OUTPUT_DIM, activation=tf.nn.tanh)
        output1 = 10*tf.layers.dense(output1, OUTPUT_DIM, activation=tf.nn.tanh)
    return tf.reshape(output+output1, [-1, OUTPUT_DIM])

def Generator_retro(n_samples, noise=None):
    if noise is None:
        noise = tf.random.uniform(shape=[n_samples], minval=0, maxval=OUTPUT_DIM, dtype=tf.dtypes.int32)
        output = tf.one_hot(noise, OUTPUT_DIM, axis=-1)
        noise1 = tf.random_normal([n_samples, OUTPUT_DIM])
    with tf.variable_scope('Retro', reuse=tf.AUTO_REUSE) as scope:
        output = tf.layers.dense(output, OUTPUT_DIM, activation=None)
        output1 = tf.layers.dense(noise1, OUTPUT_DIM, activation=tf.nn.tanh)
        output1 = 10*tf.layers.dense(output1, OUTPUT_DIM, activation=tf.nn.tanh)
    return tf.reshape(output+output1, [-1, OUTPUT_DIM])

def Copy_generators():
    i=0
    for h in gen_params:
        tf.assign(gen_params_retro[i], h)
        i=i+1


def Discriminator(inputs):
    output = tf.reshape(inputs, [-1, OUTPUT_DIM])
    
    with tf.variable_scope('Discriminator', reuse=tf.AUTO_REUSE) as scope:
        with tf.variable_scope('highway1', reuse=tf.AUTO_REUSE):
            output = highway_layer(output)
        with tf.variable_scope('highway2', reuse=tf.AUTO_REUSE):
            output = highway_layer(output)
        output = tf.nn.dropout(output, keep_prob=0.50)
        with tf.variable_scope('highway3', reuse=tf.AUTO_REUSE):
            output = highway_layer(output)
        with tf.variable_scope('highway4', reuse=tf.AUTO_REUSE):
            output = highway_layer(output)
        output = tf.nn.dropout(output, keep_prob=0.50)
        with tf.variable_scope('highway5', reuse=tf.AUTO_REUSE):
            output = highway_layer(output)
        with tf.variable_scope('highway6', reuse=tf.AUTO_REUSE):
            output = highway_layer(output)
        output = tf.math.sqrt(tf.reduce_sum(tf.square(output), axis=1)+0.0001)
    return tf.reshape(output, [-1])

P = tf.placeholder(tf.float32, shape=[OUTPUT_DIM, Dimension])
real_data = tf.placeholder(tf.float32, shape=[BATCH_SIZE, OUTPUT_DIM])
fake_data = Generator(BATCH_SIZE)
fake_data2 = Generator(BATCH_SIZE)
disc_real = Discriminator(real_data)
disc_real_= Discriminator(real_data)
disc_fake = Discriminator(fake_data)

gen_params = get_vars_from_scope('Generator')
disc_params = get_vars_from_scope('Discriminator')


fake_data_retro = Generator_retro(BATCH_SIZE)
gen_params_retro = get_vars_from_scope('Retro')

gen_cost = -tf.reduce_mean(disc_fake)
disc_cost = tf.reduce_mean(disc_fake) - tf.reduce_mean(disc_real)

#Consistency term from https://openreview.net/pdf?id=SJx9GQb0-
CT = LAMBDA_2*tf.square(disc_real-disc_real_)
CT_ = tf.maximum(CT-0.0,0.0*CT)
disc_cost += tf.reduce_mean(CT_)

# Gradient penalty
alpha = tf.random_uniform(shape=[BATCH_SIZE,1], minval=0.,maxval=1.)
differences = fake_data - real_data
interpolates = real_data + (alpha*differences)
gradients = tf.gradients(Discriminator(interpolates), [interpolates])[0]

slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), reduction_indices=[1]))
gradient_penalty = tf.reduce_mean(tf.clip_by_value(slopes - 1.0, 0, np.infty)**2)

disc_cost += LAMBDA*gradient_penalty

# I needed to know how strongly "leakage" can blow-up
leakage = LAMBDA*gradient_penalty 

Q = tf.reshape(fake_data, [BATCH_SIZE,-1])
Q2 = tf.reshape(fake_data2, [BATCH_SIZE,-1])
Gramm = tf.matmul(Q,tf.transpose(Q2))
R = tf.reshape(tf.reduce_sum(tf.multiply(Q,Q), axis=1), [BATCH_SIZE,1])
R2 = tf.reshape(tf.reduce_sum(tf.multiply(Q2,Q2), axis=1), [BATCH_SIZE,1])
t1 = tf.ones([BATCH_SIZE, 1])
t2 = tf.ones([1, BATCH_SIZE])
S = tf.multiply(tf.exp(GAMMA*(-tf.matmul(R, t2) - tf.matmul(t1, tf.transpose(R2)) + 2*Gramm)), Gramm)

Q_retro = tf.reshape(fake_data_retro, [BATCH_SIZE,-1])
R_retro = tf.reshape(tf.reduce_sum(tf.multiply(Q_retro,Q_retro), axis=1), [BATCH_SIZE,1])
Gramm2 = tf.matmul(Q,tf.transpose(Q_retro))
Q3 = tf.matmul(tf.matmul(Q_retro, P), tf.transpose(P))
Gramm3 = tf.matmul(Q,tf.transpose(Q3))
S2 = tf.multiply(tf.exp(GAMMA*(-tf.matmul(R, t2) - tf.matmul(t1, tf.transpose(R_retro)) + 2*Gramm2)), Gramm3)

gen_cost += Lambda*tf.reduce_mean(S) - 2*Lambda*tf.reduce_mean(S2)

total = tf.reduce_mean(disc_real) - tf.reduce_mean(disc_fake) + Lambda*tf.reduce_mean(S) - 2*Lambda*tf.reduce_mean(S2)


# Radam converges faster than Adam, so I use it
optimizer_gen = RAdamOptimizer(learning_rate=1e-5, beta1=0.5, beta2=0.9)
optimizer_disc = RAdamOptimizer(learning_rate=1e-5, beta1=0.5, beta2=0.9)
gen_train_op = optimizer_gen.minimize(gen_cost, var_list=gen_params)
disc_train_op = optimizer_disc.minimize(disc_cost, var_list=disc_params)


# Also, weights of discriminator are clipped as in original paper of Arjovsky et al, 
# though this is not needed in fact 
clip_ops = []
for var in disc_params:
    clip_bounds = [-10.0, 10.0]
    clip_ops.append(
        tf.assign(
            var, 
            tf.clip_by_value(var, clip_bounds[0], clip_bounds[1])
        )
    )
clip_disc_weights = tf.group(*clip_ops)


num = images.shape[0]
images = np.reshape(images, (num, -1))
data = np.random.permutation(images)

# Train loop
P_retro = np.zeros((OUTPUT_DIM,Dimension))
current = 0
c_size = int(num/BATCH_SIZE)
ft1 = np.ones([BATCH_SIZE, 1])
ft2 = np.ones([1, BATCH_SIZE])
needed_vars = gen_params + disc_params
saver = tf.train.Saver(var_list=needed_vars)
for epoch in range(num_epochs):
    with tf.Session() as session:
        session.run(tf.initialize_all_variables())
        if(epoch>0):
            saver.restore(session, "/tmp/model.ckpt")
            i=0
            for h in gen_params:
                gen_params_retro[i].load(session.run(h), session)
                i=i+1
        for iteration in range(ITERS):
            start_time = time.time()
            # Train generator
            if iteration > 0:
                start_index = current % c_size
                start_index = start_index*BATCH_SIZE
                _data = data[start_index:start_index+BATCH_SIZE]
                _ = session.run(gen_train_op, feed_dict={real_data: _data, P: P_retro})
                current = current+1
            # Train critic
            disc_iters = CRITIC_ITERS
            for i in range(disc_iters):
                start_index = current % c_size
                start_index = start_index*BATCH_SIZE
                _data = data[start_index:start_index+BATCH_SIZE]
                _disc_cost, _ = session.run([disc_cost, disc_train_op], feed_dict={real_data: _data, P: P_retro})
                _ = session.run(clip_disc_weights)
                current = current+1

            lib.plot.plot('train disc cost', _disc_cost)
            lib.plot.plot('time', time.time() - start_time)

            # Calculate dev loss and generate samples every 500 iters
            if (epoch*ITERS+iteration) % 500 == 499 or iteration == 0:
                dev_disc_costs = []
                dev_total = []
                dev_leakage = []
                for image in range(c_size):
                    start_index = image*BATCH_SIZE
                    _dev_disc_cost, _total, _leakage = session.run([disc_cost, total, leakage], feed_dict={real_data: 
                                                    data[start_index:start_index+BATCH_SIZE], P: P_retro}) 
                    dev_disc_costs.append(_dev_disc_cost)
                    dev_total.append(_total)
                    dev_leakage.append(_leakage)
                lib.plot.plot('dev disc cost', np.mean(dev_disc_costs))
                lib.plot.plot('dev total cost', np.mean(dev_total))
                lib.plot.plot('dev leakage cost', np.mean(dev_leakage))

            # Save logs every 100 iters
            if (iteration % 100 == 99): #(iteration < 2) or 
                lib.plot.flush()

            lib.plot.tick()
        M = np.zeros((OUTPUT_DIM,OUTPUT_DIM))
        for i in range(TIMES_COV):
            fake_np0 = np.reshape(session.run(fake_data), [-1, OUTPUT_DIM])
            fake_np1 = np.reshape(session.run(fake_data2), [-1, OUTPUT_DIM])
            FGramm = np.matmul(fake_np0,np.transpose(fake_np1))
            FR0 = np.reshape(np.sum(np.multiply(fake_np0,fake_np0), axis=1), [BATCH_SIZE,1])
            FR1 = np.reshape(np.sum(np.multiply(fake_np1,fake_np1), axis=1), [BATCH_SIZE,1])
            M_0 = np.exp(GAMMA*(-np.matmul(FR0, ft2) - np.matmul(ft1, np.transpose(FR1)) + 2*FGramm))
            M = M + np.matmul(np.matmul(np.transpose(fake_np0), M_0), fake_np1)
        svd = TruncatedSVD(n_components=Dimension, n_iter=7)
        svd.fit(M)
        U = svd.components_
        P_retro = np.transpose(U)
        print(svd.singular_values_)
        EV = svd.explained_variance_.sum()
        EVR = svd.explained_variance_ratio_.sum()
        Projection = np.matmul(P_retro, np.transpose(P_retro))
        DIFF = Projection - P_correct
        print("Explained variance: %f and explained variance ratio: %f" % (EV, EVR))       
        dist = np.sqrt(np.sum(DIFF*DIFF))
        lib.plot.plot('Distance till correct', dist)
        L12 = np.sum(np.sqrt(np.sum(np.square(images-np.matmul(images, Projection)), axis=1)))
        print("L12 distance %0.4f\n" %L12)
        save_path = saver.save(session, "/tmp/model.ckpt")
        session.close()